Importando as bibliotecas necessárias

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3 as lite

Criando o cursor sql que será usado posteriormente

In [ ]:
con = lite.connect('leiloeiros.db')

Adquirindo os dados do site, dando a requisição necessária

In [ ]:
res = requests.post("http://200.198.139.228/leiloeiros/busca/listar", data = {'Nome':'Todos', 'CodMunicipio':'0'})

Criando o arquivo texto em html do site

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')

Separando apenas os dados sobre os leiloeiros em lista

In [ ]:
leiloeiros = str(soup).split('<hr/>')
leiloeiros.pop()
leiloeiros.pop()
leiloeiros = leiloeiros[4:]

Criação de função que acha o intervalo de uma substring, da-se valores e retorna a substring entre esses valores:<br/>
Necessita de três argumentos:<br/>
a = string<br/>
b = substring única que esteja antes da substring que se queira<br/>
c = substring única que esteja após a substring que se queira, valor pedrão é vazio, a substring irá até o final da string<br/>
ex: "amundob" querendo-se retirar "mundo" da string, usa-se stringclip("amundob","a","b")

In [ ]:
def stringclip(a,b,c=''):
                x = str(a).find(b)
                x += len(b)
                if c != '':
                    y = str(a)[x:].find(c)
                    return str(a)[x:x+y]
                else:
                    return str(a)[x:]

Função que deleta uma substring dado os valores iniciais e finais.<br/>
a = string<br/>
b = posição inicial do corte, incluso<br/>
c = posição final do corte, incluso<br/>

In [ ]:
def delsubstr(a,b,c):
        x = str(a).find(b)
        x += len(b)
        y = str(a)[x:].find(c)
        return str(a)[x+y:]


Criando e povoando as listas que terão os dados, cada lista será uma coluna:
O padrão pra cada lista e iteração é:
- procura-se o intervalo que contém a informação da coluna e armazena
- deleta-se o intervalo com a informação e suas tags
- as colunas não utilizadas ficam em branco

In [ ]:
count = 0
matriculas = []
nomes = []
status = []
enderecos = []
sites = []
posses = []
telefones = []
emails = []
publis = []
motivos = []
prepostos = []

for i in leiloeiros:
    
    a = leiloeiros[count]

    #Procurando a matrícula:
    b = '"#A01A14">'
    c = '<'
    d = matriculas
    d.append(stringclip(a,b,c))             #adicionando a matrícula à lista
    a = delsubstr(a,b,c)                    #deletando a substring utilizada

    #Procurando o nome:
    b = ' - '
    c = '<'
    d = nomes
    d.append(stringclip(a,b,c))             #adicionando o nome à lista
    a = delsubstr(a,b,c)                    #deletando a substring utilizada
    

    if 'Cancelado' in str(a)[:50]:          #teste do status do leiloeiro, usado por terem dados diferentes
        
        status.append("Cancelado")          #Adicionando o status à lista
        b = '<font color="#FF0000">'
        c = '<br/>'
        a = delsubstr(a,b,c)                #Deletando a substring utilizada
        
        #Procurando o endereco dos cancelados
        b = '<br/>'
        c = '<br/>'
        d = enderecos               
        d.append(stringclip(a,b,c))         #Adicionando o endereco dos cancelados à lista
        a = delsubstr(a,b,c)        #       Deletando a substring utilizada

        #Procurando a data de posse dos cancelados:
        b = ': '
        c = '<'
        d = posses
        d.append(stringclip(a,b,c))         #Adicionando a data de posse dos cancelados à lista
        a = delsubstr(a,b,c)                #Deletando a substring utilizada

        #Procurando a publicacao do cancelamento:
        b = 'f="'
        c = '"'
        d = publis
        d.append(stringclip(a,b,c))         #Adicionando a publicação no diário oficial do cancelamento à lista
        a = delsubstr(a,b,c)                #Deletando a substring utilizada

        #Procurando o motivo do cancelamento:
        b = 'Motivo do Cancelamento: '
        c = ''
        d = motivos
        d.append(stringclip(a,b,c))         #Adicinando o motivo do cancelamento à lista
        a = delsubstr(a,b,c)                #Deletando a substring utilizada

        #adicionando as infos vazias:
        sites.append('')
        telefones.append('')
        emails.append('')
        prepostos.append('')

    else:

        #Foi chamado aqueles que não foram cancelados de "Atuantes"
        status.append("Atuante")            #Adicionando o status de atuante à lista

        #Procurando o site, caso houver
        if 'http://' in a[:50]:
            b = 'f="'
            c = '"'
            d = sites
            d.append(stringclip(a,b,c))     #Adicionando o site à lista
            a = delsubstr(a,b,c)            #Deletando a substring utilizada
        else:
            sites.append('')                #Adicionando o vazio à lista de site
        
        #Procurando data de posse:
        b = '</b>'
        c = '<br/>'
        d = posses
        d.append(stringclip(a,b,c))         #Adicionando a data de posse a lista
        a = delsubstr(a,b,c)                #Deletando a substring utilizada

        #Adicionando a primeira linha do endereco
        b = '<br/>'
        c = '<br/><'
        end1 = stringclip(a,b,c)            #Adicionando a primeira linha do endereco, referente a rua, será concatenada posteriormente
        end1 = end1.replace('</b>','')      #Retirando substrings extra da string que será utilizada na lista
        a = delsubstr(a,b,c)                #Deletando a substring utilizada

        #Segunda linha do endereco
        b = 'b>'
        c = '</b>'
        cidade = stringclip(a,b,c)          #Adicionando a segunda linha do endereco, referente a cidade
        cidade = cidade.replace('</b>','')  #Retirando substrings extra da string que será utilizada na lista
        a = delsubstr(a,b,c)                #Deletando a substring utilizada

        #Terceira linha do endereco retirada diretamente do str do site
        b = "<br/>"
        c = "<br/>"
        cep = stringclip(a,b,c)             #Adicionando a terceira linha do endereco, referente ao CEP
        a = delsubstr(a,b,c)                #Deletando a substring utilizada

        endereco = end1+', '+cidade+", "+cep #concatenando as strings de endereco
        enderecos.append(endereco)          #Adicionando os enderecos concatenados à lista

        #Procurando o telefone:
        b = "Telefone : "
        c = '<br/>'
        tel = stringclip(a,b,c)             #Criando a string contendo telefone
        tel = tel.replace('\xa0',' ')       #Retirando substrings extras
        telefones.append(tel)               #Adicionando o telefone à lista
        a = delsubstr(a,b,c)                #Deletando a substring utilizada


        #Procurando o email e verificando a existência de preposto:
        b = 'e-Mail : '
        c = ''

        email = stringclip(a,b,c)               #string que vai até o final

        #Verificação de preposto
        if 'Preposto' in email:
            d = "Preposto : "
            prepostos.append(stringclip(email,d))   #Adicionando o preposto a lista
            c = "<br/>"                             #Criando a var para ser usada para procurar o email
        
        else:
            prepostos.append('')                    #Adicionando o preposto vazio à lista

        emails.append(stringclip(a,b,c))            #Adicionando o email à lista
        a = delsubstr(a,b,c)                        #Deletando a substring utilizada, redundante

        #Adicionando as colunas vazias:
        publis.append('')
        motivos.append('')

    count += 1


Nome das colunas do df, já obedecendo o padrão do schema de SQL

In [ ]:
colunas = [\
            "MATRICULA",\
            "NOME",\
            "STATUS",\
            "SITE",\
            "DATA_POSSE",\
            "ENDERECOS",\
            "TELEFONES",\
            "EMAILS",\
            "PUBLICACAO_DIARIO",\
            "MOTIVO_CANCELAMENTO",\
            "PREPOSTO"
            ]

Dados que alimentarão o Dataframe

In [ ]:
dados = list(zip(\
    matriculas,\
    nomes,\
    status,\
    sites,\
    posses,\
    enderecos,\
    telefones,\
    emails,\
    publis,\
    motivos,\
    prepostos))

criando o df pandas

In [ ]:
df = pd.DataFrame(dados,columns=colunas)

Criando o db sqlite requerido por método de dataframes pandas

In [ ]:
db = df.to_sql('leiloeiros',con,index=False)